In [1]:
# !pip install focal-loss
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, Input
from sklearn.model_selection import train_test_split
import shutil
from tensorflow.keras import backend as K
from tensorflow.keras import layers, models, regularizers
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers.legacy import Adam
from focal_loss import BinaryFocalLoss

In [2]:
# glowny folder
base_dir = '/Users/milenabiernacka/Desktop/studia/DS/semestr2/PD-magisterka/Mushroom_dataset/cnn/'
# polaczenie glownego folderu i podfolderow z edible i poisonous

edible_dir = os.path.join(base_dir, 'Edible')
poisonous_dir = os.path.join(base_dir, 'Poisonous')


train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Assuming base_dir is the path to the dataset directory
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary',
    shuffle=False
)

Found 1399 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Found 301 images belonging to 2 classes.


In [3]:
model = Sequential([
    Input(shape=(150, 150, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Use linear activation for Hinge loss
])


In [4]:
def seesaw_loss_with_penalty(y_true, y_pred, penalty=0.5):
    """
    A simplified conceptual version of seesaw loss for binary classification.
    This function penalizes the loss for the majority class and reduces the penalty
    for the minority class, attempting to balance the seesaw between classes.

    Args:
    - y_true: True labels.
    - y_pred: Predicted labels.
    - penalty: The penalty factor for the majority class. Lower values reduce the loss
               contribution from the majority class.

    Returns:
    - A modified binary cross-entropy loss that incorporates class balancing.
    """
    # Standard binary cross-entropy
    bce = K.binary_crossentropy(y_true, y_pred)

    # Determine weights for each sample based on its true label
    weights = K.abs(y_true - penalty)  # Reduce weight for majority class (assumed to be 1)

    # Apply seesaw penalty
    weighted_bce = weights * bce

    return K.mean(weighted_bce)

In [5]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss=lambda y_true, y_pred: seesaw_loss_with_penalty(y_true, y_pred, penalty=0.8), metrics=['accuracy'])

In [6]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20
87/87 [==============================] - 32s 357ms/step - loss: 0.1910 - accuracy: 0.7317 - val_loss: 0.1604 - val_accuracy: 0.7431
Epoch 2/20
87/87 [==============================] - 29s 331ms/step - loss: 0.1549 - accuracy: 0.7375 - val_loss: 0.1550 - val_accuracy: 0.7361
Epoch 3/20
87/87 [==============================] - 30s 346ms/step - loss: 0.1417 - accuracy: 0.7375 - val_loss: 0.1578 - val_accuracy: 0.7326
Epoch 4/20
87/87 [==============================] - 29s 336ms/step - loss: 0.1320 - accuracy: 0.7513 - val_loss: 0.1181 - val_accuracy: 0.7604
Epoch 5/20
87/87 [==============================] - 29s 333ms/step - loss: 0.1196 - accuracy: 0.7614 - val_loss: 0.1129 - val_accuracy: 0.7986
Epoch 6/20
87/87 [==============================] - 29s 333ms/step - loss: 0.1110 - accuracy: 0.7773 - val_loss: 0.0940 - val_accuracy: 0.8264
Epoch 7/20
87/87 [==============================] - 30s 341ms/step - loss: 0.1050 - accuracy: 0.8200 - val_loss: 0.0905 - val_accuracy: 0.8194

In [7]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

18/18 [==============================] - 2s 97ms/step - loss: 0.0382 - accuracy: 0.9514
Test Loss: 0.03823953494429588, Test Accuracy: 0.9513888955116272


In [10]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss=lambda y_true, y_pred: seesaw_loss_with_penalty(y_true, y_pred, penalty=0.5), metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")